# Problemas de optimización con QAOA

Recientemente ha surgido un gran interés en utilizar computadores cuánticos para encontrar la solución de problemas de optimización combinatorial. En el siguiente taller estudiaremos como solucionar el problema $MAXCUT$ mediante QAOA.

## Problemas de optimización combinatorial

Un problema de optimización combinatorial corresponde a buscar una solución óptima en un conjunto finito o contablemente infinito de posibles soluciones, y generalmente se define con respecto a una función de coste que busca ser maximizada o minimizada.

Algunos problemas típicos de optimización combinatorial incluyen:

*  Minimización: distancias, tiempos de procesamiento, cantidad de material, consumo de energía.
*  Maximización: utilidades, eficiencia.

Cualquier problema de optimización combinatorial puede expresarse como

$$ \text{maximizar } \;\;      C(x)$$

$$ \text{sujeto a } \;\; x \in S $$

donde $x \in S$ es una variable discreta y $C : D \rightarrow \mathbb{R}$ es una función de costo, que mapea de algún dominio de $S$ a los números reales $\mathbb{R}$. La variable $x$ puede estar sujeta a un conjunto de restricciones (*constrains*) y pertenece al conjunto $S \subset D$ de posibles puntos. Si consideramos problemas de optimización combinatorial binaria, el objetivo es encontrar la cadena de n-bits $x$ para la cuál $C(x)$ es maximal.

Bajo ciertas condiciones, es posible mapear la función de costo a un Hamiltoniano que es diagonal en la base computacional y que cuente con a lo más $m$ términos locales $\hat{C}_k$, 

$$ H = \sum_{k = 1}^m \hat{C}_k = \sum_{x \in \{0,1\}^n} C(x) |x \rangle\langle x| $$

donde $x \in \{0,1\}^n$ son las etiquetas de la base computacional $|x \rangle \in \mathbb{C}^{2^n}$.

## QAOA (Quantum Approximate Optimization Algorithm)

El algoritmo QAOA (*Quantum Approximate Optimization Algorithm*) corresponde a un algortimo variacional cuántico para resolver problemas combinatoriales. QAOA utiliza una unitaria $U(\boldsymbol{\beta}, \boldsymbol{\gamma})$ caracterizada por los parámetros $(\boldsymbol{\beta}, \boldsymbol{\gamma})$ para preparar el estado cuántico $\lvert \psi(\boldsymbol{\beta}, \boldsymbol{\gamma}) \rangle$ y maximizar una función de coste $F_p(\boldsymbol{\gamma},\boldsymbol{\beta})$ que represente el problema.

La unitaria $U(\boldsymbol{\beta}, \boldsymbol{\gamma})$ esta compuesta a la vez por dos unitarias $U(\boldsymbol{\beta}) = e^{-i \boldsymbol{\beta} H_B}$ y $U(\boldsymbol{\gamma}) = e^{-i \boldsymbol{\gamma} H}$, donde $H_B$ corresponde al Hamiltoniano de mezcla (*mixing*) y $H$ corresponde al Hamiltoniano que representa el problema de optimización. Siguiendo el artículo original de Farhi et. al [1](https://arxiv.org/abs/1411.4028), el problema de optimización está codificado en un Hamiltoniano diagonal en la base computacional 

$$ H = \sum_{k = 1}^m \hat{C}_k,$$

mientras que el Hamiltoniano de mezcla corresponde a sumas de rotaciones $X$ sobre un único qubit, dado por

$$ H_B = \sum_{i = 1}^n X_i.$$

El estado objetivo del algoritmo es preparado aplicando estas unitarias en bloques alternantes $p$ veces

\begin{align}
\lvert \psi(\boldsymbol{\beta}, \boldsymbol{\gamma}) \rangle &= \underbrace{U(\boldsymbol{\beta}) U(\boldsymbol{\gamma}) 
                                            \cdots U(\boldsymbol{\beta}) U(\boldsymbol{\gamma})}_{p \; \text{veces}} 
\lvert \psi_0 \rangle \\\\
&= e^{ -i\beta_p H_B } e^{ -i\gamma_p H } \ldots e^{ -i\beta_1 H_B } e^{ -i\gamma_1 H } \lvert \psi_0 \rangle 
\end{align}

donde $\lvert \psi_0 \rangle$ es un estado inicial conveniente, utilizandose típicamente el estado producto $|+\rangle^n = \bigg(\frac{1}{\sqrt{2}}\big(\lvert 0 \rangle + \lvert 1 \rangle\big)\bigg)^{\otimes n}$.

El objetivo del algoritmo es encontrar los parámetros óptimos $(\boldsymbol{\beta}_{opt}, \boldsymbol{\gamma}_{opt})$ tales que el estado cuántico $\lvert \psi(\boldsymbol{\beta}_{opt}, \boldsymbol{\gamma}_{opt}) \rangle$ codifique la solución del problema, lo cuál corresponde a maximizar la función de coste

\begin{align}
F_p(\boldsymbol{\gamma},\boldsymbol{\beta}) &= \langle \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta})|H|\psi_p(\boldsymbol{\gamma},\boldsymbol{\beta})\rangle\\\\
&= \sum_{x \in \{0,1\}^n} C(x) |\langle x| \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta}) \rangle |^2.
\end{align}

Dado que el Hamiltoniano $H$ es diagonal en la base computacional, podemos calcular fácilmente el valor de expectación anterior muestreando el estado $| \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta}) \rangle$ en la base computacional.

Para cada cadena de bits $x$ obtenida de las distribución $|\langle x| \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta}) \rangle |^2$, evaluamos la función de coste $C(x)$ y promediamos sobre el número total de muestras.

# Problema de $MAXCUT$

El problema de *Maximum Cut* ($MAXCUT$) corresponde a un problema NP-completo, con una serie de aplicaciones en áreas como análisis de grupos, ciencia de redes y física estadística. El objetivo es particionar los nodos de un grafo en dos conjuntos, de tal forma que el número de aristas (edges) entre ambos conjuntos sea máximo. Por ejemplo, consideremos el siguiente grafo de cuatro nodos y algunas de las distintas formas en que puede ser particionado en dos conjuntos, representados por los colores azul y rojo.

<img src="img/maxcut.png"/>

Como a cada nodo se le puede asignar el conjunto azul o rojo, existen $2^4=16$ posibles elecciones, de las cuales tenemos que encontrar aquella que entrega el mayor número de aristas entre ambos conjuntos. En el ejemplo superior, el número de aristas corresponde a $0$, $2$, $2$ y $4$ de izquierda a derecha.

Si codificamos los elementos del conjunto rojo como $0$s y los elementos del conjunto azul como $1$s, es fácil ver que la cadena de bits "$0101$" y "$1010$" representan la asignación de los nodos que corresponden a la solución del problema. El objetivo de este taller será encontrar la solución de este ejemplo de $MAXCUT$ mediante QAOA en un computador cuántico.


Un ejemplo práctico en markteing del problema de $MAXCUT$ es el siguiente. Consideremos un sistema de muchas personas que pueden interactuar entre si e influenciar en las decisiones de sus vecinos más cercanos, de tal forma que los individuos pueden ser representandos como los nodos de un grafo, y interacción entre dos personas corresponde a la arista que une ambos nodos. Supongamos que esta interacción permite que los individuos influyen en las decisiones de compras de otras personas. Luego, podemos predecir el resultado de una estrategia de ventas si ofrecemos productos gratis a ciertos individuos, y después nos preguntamos cuál es el subconjunto de individuos que deben recibir las productos gratis para maximizar las utilidades.

Primero, definimos un objeto de la clase `networkx` para representar el grafo anterior.

In [ ]:
# Importamos los paquetes necesarios

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile

from qiskit.circuit import Parameter
from qiskit.tools.visualization import plot_histogram


import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])
nx.draw(G, with_labels=True, alpha=0.8, node_size=500)

# Si su grafo se ve de forma distinta (por ejemplo, con las aristas cruzadas, ejecute nuevamente esta celda de código)

El Hamiltoniano del problema $MAXCUT$ con peso esta dado por

$$
H = \sum_{i<j} w_{ij} Z_i Z_j.
$$

donde $w_{ij}$ corresponde al peso asociado a la arista que une el nodo $i$ con el nodo $j$. En este caso consideraremos pesos constantes $w_{ij} = w = 1$. Para nuestro ejemplo de cuatro nodos, vamos a necesitar un Hamiltoniano de 4 qubits dado por

$$
H = \big(Z_0 \otimes Z_1 \otimes I_2 \otimes I_3\big) + 
    \big(I_0 \otimes Z_1 \otimes Z_2 \otimes I_3\big) +
    \big(Z_0 \otimes I_1 \otimes I_2 \otimes Z_3\big) +
    \big(I_0 \otimes I_1 \otimes Z_2 \otimes Z_3\big)
$$

Por otro lado, el Hamiltoniano de mezcla es de la forma

$$
H_B = \big(X_0 \otimes I_1 \otimes I_2 \otimes I_3 \big) + 
      \big(I_0 \otimes X_1 \otimes I_2 \otimes I_3 \big) +
      \big(I_0 \otimes I_1 \otimes X_2 \otimes I_3 \big) +
      \big(I_0 \otimes I_1 \otimes I_2 \otimes X_3 \big)
$$

Como los términos individiales en la suma de $H$ y $H_B$ commutan entre ellos, podemos utilizar la fórmula de Baker-Campbell-Hausdorff para escribir las unitarias como

$$ U(H_B) = e^{-i \beta H_B} = e^{-i \beta X_0}e^{-i \beta X_1}e^{-i \beta X_2}e^{-i \beta X_3}, \quad U(H_P) = e^{-i \gamma H_P} = e^{-i \gamma Z_0 Z_1}e^{-i \gamma Z_1 Z_2}e^{-i \gamma Z_2 Z_3}e^{-i \gamma Z_0 Z_3}.$$

**Ejercicio 1:** Notando que la unitaria $U(H_B)$ corresponde a un producto de rotaciones en el eje $X$ sobre cada qubit, escriba una función que tome el párametro `beta` y entregue un circuito que implemente $U(H_B)$. Para esto, utilice la [compuerta](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html) `rx`.

In [ ]:
# El número de qubits corresponde al número de nodos del grafo

nqubits = len(G.nodes)
nqubits

In [ ]:
# Aplique una compuerta rx sobre cada qubit (el primer argumento corresponde al ángulo y el segundo argumento corresponde al qubit)


#def unitaria_HB(beta):
    # Escriba su codigo aqui
    
    
    
# Solución
    
def unitaria_HB(beta):
    qc_HB = QuantumCircuit(nqubits)
    for i in range(0, nqubits):
        qc_HB.rx(2 * beta, i)
  
    return qc_HB

In [ ]:
# Utilizaremos la función Parameter para asignar un parámetro arbitrario "beta"

beta = Parameter('β')

U_HB = unitaria_HB(beta)
U_HB.draw('mpl')

**Ejercicio 2:** Notamos que la unitaria $U(H)$ esta compuesta de productos de unitarias del tipo 
$$U(\theta) = e^{i \theta Z \otimes Z} = \left( \matrix{ e^{i \theta} & 0 & 0 & 0 \cr
0 & e^{-i \theta} & 0 & 0 \cr
0 & 0 & e^{-i \theta} & 0 \cr
0 & 0 & 0 & e^{i \theta} \cr} \right),$$

las cuales pueden ser construidas usando la [compuerta](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZZGate.html) `rzz()` de qiskit. Defina una función que tome el párametro `beta` y entregue un circuito que implemente $U(H)$.

**Observación**: Recuerde que la conectividad entre qubits debe ser igual que las aristas del grafo.

In [ ]:
# Aristas del grafo

list(G.edges())

In [ ]:
# Aplique una compuerta rzz sobre cada par de qubits de la lista anterior
# El primer argumento corresponde al ángulo, el segundo argumento corresponde al qubit de control y el tercer corresponde al qubit objetivo

#def unitaria_H(gamma):
    # Escriba su código aquí

# Solución

def unitaria_H(gamma):
    qc_H = QuantumCircuit(nqubits)
    for pair in list(G.edges()):
        qc_H.rzz(2 * gamma, pair[0], pair[1])

    return qc_H


def unitaria_H2(gamma):
    qc_H = QuantumCircuit(nqubits)
    
    qc_H.rzz(2 * gamma, 0, 1)
    qc_H.rzz(2 * gamma, 0, 3)
    qc_H.rzz(2 * gamma, 1, 2)
    qc_H.rzz(2 * gamma, 2, 3)

    return qc_H

In [ ]:
gamma = Parameter('γ')

U_H = unitaria_H(gamma)
U_H.draw('mpl')

**Ejercicio 3:** Defina una función que construya el estado inicial $|+\rangle^{\otimes 4} = |+\rangle|+\rangle|+\rangle|+\rangle = \bigg(\frac{1}{\sqrt{2}}\big(\lvert 0 \rangle + \lvert 1 \rangle\big)\bigg)^{\otimes 4}$.

In [ ]:
# Aplique una compuerta Hadamard sobre cada qubit

#def estado_inicial(nqubits=nqubits):
    # Escriba su codigo aquí


# Solución
    
def estado_inicial(nqubits=nqubits):
    qc_ini = QuantumCircuit(nqubits)
    for i in range(nqubits):
        qc_ini.h(i)

    return qc_ini

In [ ]:
estado_0 = estado_inicial()
estado_0.draw('mpl')

Mediante las tres funciones definidas anteriormente, podemos construir un circuito que implemente el estado cuántico $|\psi(\boldsymbol{\beta}, \boldsymbol{\gamma}) \rangle$.

**Ejercicio 4:** Construya una función que reciba como entrada el estado inicial $| \psi_0 \rangle$ y las unitarias $U(H)$ y $U(H_B)$, y entregue un circuito que implemente el estado cuántico $|\psi(\boldsymbol{\beta}, \boldsymbol{\gamma}) \rangle$.

In [ ]:
# Usando compose(), combine las tres funciones definidas anteriormente para implementar el estado psi

#def circuito_QAOA(estado_0, U_H, U_HB):
    # Escriba su código aquí
    
    
    
# Solución


def circuito_QAOA(estado_0, U_H, U_HB):
    qc_qaoa = QuantumCircuit(nqubits)
    qc_qaoa.compose( estado_0, [i for i in range(0, nqubits)], inplace=True )
    qc_qaoa.barrier()
    qc_qaoa.compose( U_H, [i for i in range(0, nqubits)], inplace=True )
    qc_qaoa.barrier()
    qc_qaoa.compose( U_HB, [i for i in range(0, nqubits)], inplace=True )
  
    return qc_qaoa

In [ ]:
qc_qaoa = circuito_QAOA(estado_0, U_H, U_HB)

qc_qaoa.decompose(reps=0).draw('mpl')

Ahora que podemos construir el estado $|\psi(\boldsymbol{\beta}, \boldsymbol{\gamma}) \rangle$, debemos buscar una forma de calcular la función de coste

\begin{align}
F_p(\boldsymbol{\gamma},\boldsymbol{\beta}) &= \langle \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta})|H|\psi_p(\boldsymbol{\gamma},\boldsymbol{\beta})\rangle\\\\
&= \sum_{x \in \{0,1\}^n} C(x) |\langle x| \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta}) \rangle |^2.
\end{align}
y optimizar sobre los parámetros para encontrar la solución óptima $(\boldsymbol{\beta}_{opt}, \boldsymbol{\gamma}_{opt})$.

**Ejercicio 5:** Utilizando las funciones anteriores, construya una función que mida el circuito en la base compuitacional para obtener $|\langle x| \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta}) \rangle |^2$, usando el simulador de `Aer`.

La función debe tener como entrada los ángulos $(\boldsymbol{\gamma},\boldsymbol{\beta})$ y el número de *shots* para simular $|\langle x| \psi_p(\boldsymbol{\gamma},\boldsymbol{\beta}) \rangle |^2$, y debe entregar el número de cuentas asociadas a cada cadena de bits $x$.

In [ ]:
# Para resolver este ejercicio, debe realizar los siguientes pasos 
# (si no recuerda bien como aplicar algun paso, puede revisar el taller anterior)

# - Defina un backend usando Aer.get_backend('aer_simulator')
# - Defina las unitarias U(H) y U(H_B), que dependen de gamma y beta respectivamente
# - Cree un circuito usando la función definida en el ejercicio anterior y MIDA
# - Transpile su circuito mediante la función transpile(qc, backend) 
# - Ejecute el circuito, usando la función backend.run(qc, shots=shots).result()
# - Obtenga las cuentas aplicando el método .get_counts() sobre el resultado anterior


#def medicion(gamma, beta, shots):
    # Escriba su código aquí
    
    
    
def medicion(gamma, beta, shots):

    backend = Aer.get_backend('aer_simulator')

    UH = unitaria_H(gamma)
    UHB = unitaria_HB(beta)

    qc_qaoa = circuito_QAOA(estado_0, UH, UHB)
    qc_qaoa.measure_all()

    qc_qaoa = transpile(qc_qaoa, backend)

    result = backend.run(qc_qaoa, shots=shots).result()
    counts = result.get_counts()

    return counts

Si tomamos por ejemplo los ángulos $\boldsymbol{\gamma} = \boldsymbol{\beta} = 10$, obtenemos las siguientes cuentas

In [ ]:
medicion(gamma=10, beta=10, shots=100)

Dada una cadena de bits como posible solución, necesitamos una función que entregue el número de aristas que es compartida por ambas particiones del gráfico $C(x)$. Esto lo podemos lograr con la siguiente función:

In [ ]:
def maxcut_obj(x, G):
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
            
    return obj

Por ejemplo, si tomamos la cadena de bits '$1000$' (primer nodo azul y el resto rojo), entonces hay 2 aristas que son compartidas por ambas particiones.


In [ ]:
# El resultado es negativo, ya que más adelante usaremos una función de minimización en vez de maximización

maxcut_obj('1000', G)

Finalmente, necesitamos realizar post-procesamiento para determinar la función de coste $F_p(\boldsymbol{\gamma},\boldsymbol{\beta})$ a partir de los resultados de medición.

In [ ]:
def funcion_coste(counts, G):
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():

        obj = maxcut_obj(bitstring[::-1], G)     # calcula C(x) para una cadena de bits x y grafo G, 
                                                 # el indexado [::-1] invierte el orden de los bits
        avg += obj * count                       # calcula  Σ_x C(x) * |< x | psi >|ˆ2
        sum_count += count

    return avg/sum_count                         # promedia por el número de cuentas (normalizar)

Considerando nuevamente los ángulos $\boldsymbol{\gamma} = \boldsymbol{\beta} = 10$, obtenemos el siguiente valor de la función de costo.

In [ ]:
counts_ej = medicion(gamma=10, beta=10, shots=100)

funcion_coste(counts_ej, G)

**Ejercicio 6:** A partir de las funciones anteriores, construya una función que reciba los párametros de optimización $\boldsymbol{\theta} = (\boldsymbol{\gamma}, \boldsymbol{\beta})$, el número de shots y el grafo $G$, y entregue el valor de la función de costo.

Observación: $\boldsymbol{\theta}$ corresponde a una lista, donde el primer término es $\boldsymbol{\gamma}$ y el segundo término es $\boldsymbol{\beta}$.

In [ ]:
# Para este ejercicio debe utilizar las funciones definidas anteriormente
# Note que theta tiene dos elementos, tal que gamma = theta[0] y beta = theta[1]

#def algoritmo_QAOA(theta, G, shots):
    # Escriba su código aquí
    
    
# Solución

def algoritmo_QAOA(theta, G, shots):

    gamma = theta[0]
    beta = theta[1]

    counts = medicion(gamma, beta, shots)
    coste = funcion_coste(counts, G)
    
    return coste

Podemos utilizar la función `minimize` del paquete `scipy` para llevar a cabo el proceso de optimización y encontrar los párametros óptimos, es decir, aquellos que minimizan la energía.

In [ ]:
from scipy.optimize import minimize

shots = 8192


# la función minimize recibe una función que tenga un solo argumento,
# por lo que utilizamos una función lambda para que dependa sólo de los parámetros

costo_QAOA = lambda theta: algoritmo_QAOA(theta, G=G, shots=shots)

theta_ini = [1.0, 1.0]

# utilizamos el método de optimización COBYLA

results = minimize(costo_QAOA, theta_ini, method='COBYLA')

# Reultados de Optimización

results

In [ ]:
# Parámetros óptimos

results.x

**Ejercicio 7:** Grafique las cuentas obtenidas al ejecutar el circuito con los parámetros óptimos utilizando $8192$ *shots*

In [ ]:
# Utilice la función medicion, usando el resultado anterior como argumentos
# Luego, realice un grafico de las cuentas


# Solución

counts = medicion(results.x[0], results.x[1], shots)
plot_histogram(counts)

Notamos que obtenemos con una alta probabilidad los resultados correctos. Podemos condensar las funciones de los ejercicios anteriores en una sola rutina que realice la optimización para cualquier grafo, y además agregar la opción de considerar un mayor número de repeticiones ($p$).

In [ ]:
def circuito_QAOA_gen(G, theta):
    
    nqubits = len(G.nodes())             # número de qubits
    p = len(theta)//2                    # p bloques alternantes según el tamaño de theta
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]                     # parámetros beta
    gamma = theta[p:]                    # parámetros gamma
    
    # estado inicial |+>

    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # unitaria del problema
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

        # unitaria de mezcla
        for i in range(0, nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
    return qc


def algoritmo_QAOA_gen(G, shots):
    
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def ejecutar_circuito(theta):
        
        qc = circuito_QAOA_gen(G, theta)
        counts = backend.run(qc, shots=shots).result().get_counts()
        
        return funcion_coste(counts, G)
    
    return ejecutar_circuito

A continuación, usaremos la función `random_regular_graph` de `networkx` para generar un grafo aleatorio con el cuál probar nuestro código.

In [ ]:
n = 6     # número de nodos
d = 3     # número de aristas por nodo

Gn = nx.random_regular_graph(d, n, seed=100)
pos = nx.spring_layout(Gn)
nx.draw(Gn, with_labels=True, alpha=0.8, node_size=500, pos=pos, nodelist=range(n))

Creamos una función que resuelva el problema mediante fuerza bruta para comparar la calidad de nuestra solución obtenida mediante QAOA.

In [ ]:
def fuerza_bruta(Gn):
    Gn_bf = Gn
    nx.set_edge_attributes(Gn_bf, values = 1, name = 'weight')
    n = Gn_bf.number_of_nodes()
    
    w = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            temp = Gn_bf.get_edge_data(i, j, default=0)
            if temp != 0:
                w[i, j] = temp["weight"]
    
    best_cost_brute = 0
    solutions = dict()
    
    for b in range(2**n):
        string = bin(b)[2:].zfill(n)
        x = [int(t) for t in reversed(list(string))]
        cost = 0
        for i in range(n):
            for j in range(n):
                cost = cost + w[i, j] * x[i] * (1 - x[j])
        solutions[string] = cost
    
    max_cost = max(solutions.values())
    best_solutions = [k for k,v in solutions.items() if v == max_cost]
    
    sol_vectors = []
    for i in range(len(best_solutions)):
        sol_vector_i = [int(t) for t in list(best_solutions[i])]
        sol_vectors.append(sol_vector_i)
        
    print('Mejor coste = ' +str(max_cost))
    print(best_solutions)
    
    return sol_vectors

In [ ]:
# Función para dibujar la solución del grafo

def draw_graph(G, solution_vector):
    default_axes = plt.axes(frameon=True)
    n = len(G.nodes)
    colors = ["r" if solution_vector[i] == 0 else "c" for i in range(n)]
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, node_color=colors, node_size=500, alpha=0.8, ax=default_axes, pos=pos, nodelist=range(n))

In [ ]:
# solucion mediante fuerza bruta

solution_fb = fuerza_bruta(Gn)
draw_graph(Gn, solution_fb[0])

In [ ]:
# Resolvemos el problema de MAXCUT mediante nuestra rutina

shots = 10000
costo_QAOA = algoritmo_QAOA_gen(Gn, shots)

# número de bloques p

# p = 1
p = 8

theta_ini = [1.0, 1.0]*p


results = minimize(costo_QAOA, theta_ini, method='COBYLA')
results

In [ ]:
backend = Aer.get_backend('aer_simulator')
qc_res = circuito_QAOA_gen(Gn, results.x)
counts = backend.run(qc_res, shots=shots).result().get_counts()

# ploteamos solamente las cadenas de bits con mayor número de cuentas

plot_histogram(counts, figsize=(10, 5), number_to_keep=15, sort='value_desc')  

En caso de tener un gráfo de mayor complejidad, podemos aumentar el valor de $p$ para obtener un resultado de mejor calidad, pero con la desventaja de aumentar la profundidad del circuito.

In [ ]:
print(circuito_QAOA_gen(Gn, theta_ini).depth())
circuito_QAOA_gen(Gn, theta_ini).draw('mpl')

Los ejercicios anteriores nos permitieron implementar QAOA para el problema de $MAXCUT$ de forma artesanal. Sin embargo, Qiskit cuenta con herramientas que permiten crear una instancia `Maxcut`, lo cuál permite resolver directamente el problema con la clase `QAOA` del paquete `qiskit.algorithms`.

In [ ]:
# Primero importamos los paquetes y funciones necesarias

from qiskit_optimization.applications import Maxcut
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms.eigensolvers import NumPyEigensolver
from qiskit.algorithms.minimum_eigensolvers import QAOA

In [ ]:
n = 7     # número de nodos
d = 4     # número de aristas por nodo

Gn = nx.random_regular_graph(d, n, seed=6)

nx.draw(Gn, with_labels=True, alpha=0.8, node_size=500, pos = nx.spring_layout(Gn))

In [ ]:
# solucion mediante fuerza bruta

solution_fb = fuerza_bruta(Gn)
draw_graph(Gn, solution_fb[0])

Para resolver el problema de optimización, primero es necesario definir una instancia $MAXCUT$ y convertir el problema en un programa cuadrático. Dicho programa expresa el problema de optimización como una función objetivo cuadrática, utilizando $n$ variables binarias $x_{i,j}$ (variables que toman los valores $0$ o $1$), donde $n$ es el número de nodos.

In [ ]:
# Creamos una instancia Maxcut a partir del grafo, y lo convertimos en un programa cuadrático

max_cut = Maxcut(Gn)
qp = max_cut.to_quadratic_program()
print(qp.prettyprint())

Para resolver el problema de $MAXCUT$, debemos convertir el programa cuadrático en un problema sobre el cuál pueda trabajar el computador cuantico. Para esto, convertimos el problema cuadrático en un QUBO (*Quadratic Unconstrained Binary Optimization*), a partir del cual podemos obtener un Hamiltoniano de Ising, el cual permite representar nuestro problema de tal forma que el estado de minima energía represente la solución del problema de optimización.

In [ ]:
qp2qubo = QuadraticProgramToQubo()           # Creamos una instancia de la clase QUBO
qubo = qp2qubo.convert(qp)                   # Convertimos el programa cuadrático en un QUBO
qubitOp, offset = qubo.to_ising()            # Convertimos el QUBO en un Hamiltoniano de Ising

print("Offset:", offset)
print("Hamiltoniano de Ising:")
print(str(qubitOp))

Para verificar que este Hamiltoniano nos permita encontrar la solución del problema, podemos obtendremos el autovalor de mínima energía del Hamiltoniano mediante un cálculo exacto usando la rutina clásica `NumpyEigensolver`. Notamos que existen distintas soluciones posibles al problema de $MAXCUT$, por lo que buscaremos los $k$ autovalores de mínima energa.


Observación: Esta rutina no es eficiente en el número de qubits, por lo que solo funciona para un número bajo de qubits (por eso estamos interesados en resolver el problema usando un computador cuántico!).

In [ ]:
# Variamos el valor de k dependiendo de que tan degenerado es el autoestado de mínima energía

k = 2
exact = NumPyEigensolver(k=k)
result = exact.compute_eigenvalues(qubitOp)
print(result.eigenvalues.real)

In [ ]:
# A partir del autoestado, obtenemos el vector que representa la solución del problema de Maxcut usando la función sample_most_likely

solution_vectors = []

for i in range(k):
    result_bitstring = max_cut.sample_most_likely(result.eigenstates[i].data)
    solution_vectors.append(result_bitstring)
    print(result_bitstring)

In [ ]:
draw_graph(Gn, solution_vectors[0])

**Ejercicio 8 (Tarea):** Utilizando la [rutina](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.QAOA.html) `QAOA` de qiskit, obtenga la mejor solución que pueda del problema de $MAXCUT$ para el grafo definido anteriormente. Esta función tiene las siguientes entradas (tiene más, pero nos concentraremos en estas):

- Un `Sampler`, el cuál muestrea nuestro circuito para obtener las cuentas al ejecutar el circuito en forma de cuasi-distribuciones. Para especificar el número de shots, se utiliza el argumento `options = {'shots': shots}`.

- Un [optimizador clásico](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.html#local-optimizers) (`optimizer`).

- El número de bloques alternantes (`reps`).

- El estado inicial (`initial_state`).

Observación: Para obtener el resultado, utilice el método `QAOA.compute_minimum_eigenvalue(qubitOp)`.

Puede modificar el número de nodos `n` y el número de aristas por nodo `d` para probar distintos grafos!

In [ ]:
# Exportamos los paquetes necesarios

from qiskit.algorithms.optimizers import *
from qiskit.primitives import Sampler


# Definimos el estado inicial

qc_ini = estado_inicial(n)


# Definimos el Sampler con un cierto número de shots

shots = 10000                                        # puede variar este número a gusto
sampler = Sampler(options = {'shots': shots})


# Definimos el optimizador clásico (usando ciertos parámetros) y el número de repeticiones


# Solución


# Definimos el numero de repeticiones p

reps = 20

# Definimos el optimizador clásico
optimizer = COBYLA()

In [ ]:
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=reps, initial_state=qc_ini)
result_qaoa = qaoa.compute_minimum_eigenvalue(qubitOp)

In [ ]:
# Coste encontrado

result_qaoa.eigenvalue

In [ ]:
# Grafico de las cuasi-probabilidades

plot_histogram(result_qaoa.eigenstate, figsize=(10, 5), number_to_keep=15, sort='value_desc')

La rutina de QAOA de qiskit entrega tanto el autovalor de mínima energía del Hamiltoniano de Ising junto a su correspondiente autoestado  como una cuasi-distribución de probabilidad. Para interpretar dicho resultado, transformaremos el autoestado a una lista que representa el vector solución del problema.

In [ ]:
from qiskit.result import QuasiDistribution
from qiskit.quantum_info import Statevector
from __future__ import annotations

def bitfield(n: int, L: int) -> list[int]:
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]

def sample_most_likely(state_vector: QuasiDistribution | Statevector) -> np.ndarray:
 
    if isinstance(state_vector, QuasiDistribution):
        values = list(state_vector.values())
    else:
        values = state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)

In [ ]:
sol_opt_qaoa = sample_most_likely(result_qaoa.eigenstate)
sol_opt_qaoa

In [ ]:
draw_graph(Gn, sol_opt_qaoa)

# Referencias

[[1]](https://arxiv.org/abs/1411.4028) Farhi, Edward, Jeffrey Goldstone, and Sam Gutmann. "A quantum approximate optimization algorithm." arXiv preprint arXiv:1411.4028 (2014).

[Qiskit Textbook Chapter 4](https://qiskit.org/textbook/ch-applications/qaoa.html)

[Qiskit Tutorial 05](https://qiskit.org/documentation/tutorials/algorithms/05_qaoa.html)

[Qiskit Optimization Tutorial 06](https://qiskit.org/documentation/optimization/tutorials/06_examples_max_cut_and_tsp.html)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table